In [1]:
import heapq
from typing import List, Tuple

class EightPuzzle:
    def __init__(self, initial_state: List[List[int]]):
        """
        Initialize the 8-puzzle problem
        
        Args:
            initial_state: 3x3 grid representing the initial puzzle state
        """
        self.initial_state = initial_state
        self.goal_state = [
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]  # 0 represents the empty tile
        ]
    
    def get_blank_position(self, state: List[List[int]]) -> Tuple[int, int]:
        """
        Find the position of the blank (0) tile
        
        Args:
            state: Current puzzle state
        
        Returns:
            Tuple of (row, col) for the blank tile
        """
        for r in range(3):
            for c in range(3):
                if state[r][c] == 0:
                    return r, c
        raise ValueError("No blank tile found")
    
    def get_possible_moves(self, state: List[List[int]]) -> List[List[List[int]]]:
        """
        Generate possible moves by moving the blank tile
        
        Args:
            state: Current puzzle state
        
        Returns:
            List of possible new states after moving the blank tile
        """
        moves = []
        directions = [
            (0, 1),   # right
            (0, -1),  # left
            (1, 0),   # down
            (-1, 0)   # up
        ]
        
        blank_r, blank_c = self.get_blank_position(state)
        
        for dr, dc in directions:
            new_r, new_c = blank_r + dr, blank_c + dc
            
            # Check if the new position is valid
            if 0 <= new_r < 3 and 0 <= new_c < 3:
                # Create a deep copy of the state
                new_state = [row[:] for row in state]
                
                # Swap blank tile with the adjacent tile
                new_state[blank_r][blank_c], new_state[new_r][new_c] = \
                new_state[new_r][new_c], new_state[blank_r][blank_c]
                
                moves.append(new_state)
        
        return moves
    
    def calculate_heuristic(self, state: List[List[int]]) -> int:
        """
        Calculate Manhattan distance heuristic
        
        Args:
            state: Current puzzle state
        
        Returns:
            Heuristic value (total Manhattan distance)
        """
        distance = 0
        for r in range(3):
            for c in range(3):
                if state[r][c] != 0:
                    goal_r = (state[r][c] - 1) // 3
                    goal_c = (state[r][c] - 1) % 3
                    distance += abs(r - goal_r) + abs(c - goal_c)
        return distance
    
    def best_first_search(self) -> List[List[List[int]]]:
        """
        Solve the 8-puzzle using Best First Search
        
        Returns:
            Solution path from initial state to goal state
        """
        # Priority queue to store states
        # Each item is (heuristic, state, path)
        pq = [(self.calculate_heuristic(self.initial_state), 
               self.initial_state, 
               [self.initial_state])]
        
        # Set to keep track of visited states to avoid cycles
        visited = set(tuple(map(tuple, self.initial_state)))
        
        while pq:
            _, current_state, path = heapq.heappop(pq)
            
            # Check if current state is the goal state
            if current_state == self.goal_state:
                return path
            
            # Generate possible moves
            for move in self.get_possible_moves(current_state):
                # Convert move to hashable type for visited check
                move_tuple = tuple(map(tuple, move))
                
                # Skip if state has been visited
                if move_tuple not in visited:
                    visited.add(move_tuple)
                    
                    # Calculate heuristic and add to priority queue
                    heuristic = self.calculate_heuristic(move)
                    heapq.heappush(pq, (heuristic, move, path + [move]))
        
        return []  # No solution found
    
    def print_solution(self, solution: List[List[List[int]]]):
        """
        Print the solution path
        
        Args:
            solution: List of states from initial to goal
        """
        if not solution:
            print("No solution found.")
            return
        
        print("Solution Path:")
        for i, state in enumerate(solution):
            print(f"Step {i}:")
            for row in state:
                print(row)
            print()

# Example usage
def main():
    # Example initial state
    initial_state = [
        [1, 2, 3],
        [4, 0, 6],
        [7, 5, 8]
    ]
    
    puzzle = EightPuzzle(initial_state)
    solution = puzzle.best_first_search()
    puzzle.print_solution(solution)

if __name__ == "__main__":
    main()

Solution Path:
Step 0:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 1:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [1]:
# using a star
import heapq
from typing import List, Tuple

class AStarPuzzleSolver:
    def __init__(self, initial_board: List[List[int]]):
        """
        Initialize the A* puzzle solver
        
        Args:
            initial_board: 3x3 grid representing the initial puzzle configuration
        """
        self.initial_board = initial_board
        self.target_board = [
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]  # 0 represents the empty tile
        ]
    
    def find_empty_tile_position(self, board_state: List[List[int]]) -> Tuple[int, int]:
        """
        Find the position of the blank (0) tile
        
        Args:
            board_state: Current puzzle configuration
        
        Returns:
            Tuple of (row, column) for the blank tile
        """
        for row_idx in range(3):
            for col_idx in range(3):
                if board_state[row_idx][col_idx] == 0:
                    return row_idx, col_idx
        raise ValueError("No empty tile found in the board")
    
    def generate_possible_configurations(self, board_state: List[List[int]]) -> List[List[List[int]]]:
        """
        Generate possible board configurations by moving the empty tile
        
        Args:
            board_state: Current puzzle configuration
        
        Returns:
            List of possible new board configurations
        """
        possible_configurations = []
        movement_directions = [
            (0, 1),   # right
            (0, -1),  # left
            (1, 0),   # down
            (-1, 0)   # up
        ]
        
        empty_row, empty_col = self.find_empty_tile_position(board_state)
        
        for delta_row, delta_col in movement_directions:
            new_row, new_col = empty_row + delta_row, empty_col + delta_col
            
            # Check if the new position is valid
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                # Create a deep copy of the board
                new_board = [row[:] for row in board_state]
                
                # Swap empty tile with the adjacent tile
                new_board[empty_row][empty_col], new_board[new_row][new_col] = \
                new_board[new_row][new_col], new_board[empty_row][empty_col]
                
                possible_configurations.append(new_board)
        
        return possible_configurations
    
    def calculate_manhattan_distance(self, board_state: List[List[int]]) -> int:
        """
        Calculate Manhattan distance heuristic
        
        Args:
            board_state: Current puzzle configuration
        
        Returns:
            Heuristic value (total Manhattan distance)
        """
        total_distance = 0
        for row_idx in range(3):
            for col_idx in range(3):
                if board_state[row_idx][col_idx] != 0:
                    target_row = (board_state[row_idx][col_idx] - 1) // 3
                    target_col = (board_state[row_idx][col_idx] - 1) % 3
                    total_distance += abs(row_idx - target_row) + abs(col_idx - target_col)
        return total_distance
    
    def solve_puzzle_a_star(self) -> List[List[List[int]]]:
        """
        Solve the puzzle using A* Search Algorithm
        
        Returns:
            Solution path from initial board to target board
        """
        # Priority queue to store board states
        # Each item is (f_score, g_score, board_state, solution_path)
        search_queue = [(
            self.calculate_manhattan_distance(self.initial_board),  # f_score
            0,  # g_score (initial cost)
            self.initial_board, 
            [self.initial_board]
        )]
        
        # Set to track explored board configurations
        explored_configurations = set(tuple(map(tuple, self.initial_board)))
        
        while search_queue:
            # Extract board with lowest f_score
            _, current_path_cost, current_board, solution_path = heapq.heappop(search_queue)
            
            # Check if current board matches target configuration
            if current_board == self.target_board:
                return solution_path
            
            # Generate possible board moves
            for next_board in self.generate_possible_configurations(current_board):
                # Convert board to hashable type
                board_signature = tuple(map(tuple, next_board))
                
                # Calculate path cost (g_score)
                next_path_cost = current_path_cost + 1
                
                # Calculate heuristic (h_score)
                heuristic_cost = self.calculate_manhattan_distance(next_board)
                
                # Calculate total f_score
                total_f_score = next_path_cost + heuristic_cost
                
                # Skip if board configuration already explored
                if board_signature not in explored_configurations:
                    explored_configurations.add(board_signature)
                    
                    # Add to search queue
                    heapq.heappush(search_queue, (
                        total_f_score,  # f_score = g_score + h_score
                        next_path_cost,  # g_score (cost to reach this state)
                        next_board, 
                        solution_path + [next_board]
                    ))
        
        return []  # No solution found
    
    def display_solution_steps(self, solution_steps: List[List[List[int]]]):
        """
        Print the solution path
        
        Args:
            solution_steps: List of board configurations from initial to target
        """
        if not solution_steps:
            print("No solution found.")
            return
        
        print("Solution Path:")
        for step_number, board_configuration in enumerate(solution_steps):
            print(f"Step {step_number}:")
            for row in board_configuration:
                print(row)
            print()

# Example usage
def main():
    # Example initial board configuration
    initial_board = [
        [1, 2, 0],
        [4, 6, 3],
        [7, 5, 8]
    ]
    
    puzzle = AStarPuzzleSolver(initial_board)
    solution = puzzle.solve_puzzle_a_star()
    puzzle.display_solution_steps(solution)

if __name__ == "__main__":
    main()


Solution Path:
Step 0:
[1, 2, 0]
[4, 6, 3]
[7, 5, 8]

Step 1:
[1, 2, 3]
[4, 6, 0]
[7, 5, 8]

Step 2:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 3:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 4:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

